In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [97]:
df = pd.read_csv('data/Loan Eligibility Prediction.csv')
df.head()

,Customer_ID,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,569,Female,No,0,Graduate,No,2378,0.0,9,360,1,Urban,N
1,15,Male,Yes,2,Graduate,No,1299,1086.0,17,120,1,Urban,Y
2,95,Male,No,0,Not Graduate,No,3620,0.0,25,120,1,Semiurban,Y
3,134,Male,Yes,0,Graduate,Yes,3459,0.0,25,120,1,Semiurban,Y
4,556,Male,Yes,1,Graduate,No,5468,1032.0,26,360,1,Semiurban,Y


## Encoding Variables

In [98]:
loan_ml = df.copy()
binary_map = {
    'Gender': {'Male': 1, 'Female': 0},
    'Married': {'Yes': 1, 'No': 0},
    'Education': {'Graduate': 1, 'Not Graduate': 0},
    'Self_Employed': {'Yes': 1, 'No': 0},
    'Loan_Status': {'Y': 1, 'N': 0}
}

loan_ml.replace(binary_map, inplace=True)
loan_ml = loan_ml.drop(['Customer_ID'],axis=1).copy()

C:\Users\Caleb\AppData\Local\Temp\ipykernel_15692\898801625.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  loan_ml.replace(binary_map, inplace=True)


In [99]:
loan_ml = pd.get_dummies(loan_ml, columns=['Property_Area'], drop_first=True)
loan_ml.head()

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Semiurban,Property_Area_Urban
0,0,0,0,1,0,2378,0.0,9,360,1,0,False,True
1,1,1,2,1,0,1299,1086.0,17,120,1,1,False,True
2,1,0,0,0,0,3620,0.0,25,120,1,1,True,False
3,1,1,0,1,1,3459,0.0,25,120,1,1,True,False
4,1,1,1,1,0,5468,1032.0,26,360,1,1,True,False


In [100]:
from sklearn.model_selection import train_test_split

X = loan_ml.drop(columns=['Loan_Status'])
y = loan_ml['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Logistic Model

In [102]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(class_weight='balanced',max_iter=1000)
log_model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [105]:
from sklearn.metrics import accuracy_score

y_pred = log_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8048780487804879


In [106]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.58      0.65        38
           1       0.83      0.91      0.87        85

    accuracy                           0.80       123
   macro avg       0.78      0.74      0.76       123
weighted avg       0.80      0.80      0.80       123

[[22 16]
 [ 8 77]]
